# Preprocess data

## Step 0: Make yearly files
Data is stored separately for each tile. Combine them to a yearly files

In [ ]:
import glob
import h5py as hf
import numpy as np
import pandas as pd
import config as cfg
import utils_ls as lut

data_path = 'data/ls7/peryear_pertile/'
dstination_path = "data/ls7/after_qa/"
years = cfg.years

# load hdfs and csvs

hdfs = sorted(glob.glob(data_path+'*.hdf'))

csvs = sorted(glob.glob(data_path+'*.csv'))

# for each year collect data from different files
for year in cfg.years:
    # new yearly hdf file
    newcsv = dstination_path+'attributes_'+str(year)+'.csv'
    newhdf = dstination_path+'barley_'+str(year)+'.hdf'
    print(year)

    # create new empty yearly-csv
    idx_files_to_combine = np.flatnonzero(np.core.defchararray.find(csvs, str(year)) != -1)
    new_df = pd.read_csv(csvs[0], nrows=0, index_col=False)
    with hf.File(newhdf, 'w') as f:
        for idx in idx_files_to_combine:
            csvi = csvs[idx]
            hdfi = hdfs[idx]

            # read attribute df of a tile
            df = pd.read_csv(csvi, index_col=False)

            # filter df rows based on hdf keys
            with hf.File(hdfi, 'r') as rf:
                keys = list(rf.keys())
            keys_int = np.sort(np.array([int(x) for x in keys]))
            df = df[df.new_ID.isin(keys_int)].reset_index()

            # read the  dataset from tile-hdf and write to yearly-hdf
            with hf.File(hdfi, 'r') as fi:
                for key in df.new_ID.values:
                    im = fi[str(key)][:]
                    f.create_dataset(str(key), data=im, dtype='i2',
                                     compression="gzip", compression_opts=9)

            new_df = pd.concat([new_df, df], ignore_index=True, sort=False)
    new_df.reset_index(inplace=True, drop=True)
    new_df[new_df.columns[10:]] = new_df[new_df.columns[10:]].astype(bool)
    new_df['year'] = new_df['year'].astype(int)
    new_df['area'] = new_df['area'].astype(int)
    new_df['loss'] = new_df['loss'].astype(int)
    new_df['new_ID'] = new_df['new_ID'].astype(int)
    new_df['orig_ID'] = new_df['orig_ID'].astype(str)
    if 0:
        new_df.to_csv(newcsv, index=False)


## Step 1: Create median field from yearly files that were created in step 0

In [2]:
import glob
import h5py as hf
import numpy as np
import pandas as pd
import config as cfg
import utils_ls as lut

hdfs = sorted(glob.glob('data/ls7/after_qa/*.hdf'))
csvs = sorted(glob.glob('data/ls7/after_qa/*.csv'))
destination_path = 'data/ls7/median/'

nfiles = len(hdfs)
for j in np.arange(nfiles):
    hdf = hdfs[j]
    csv = csvs[j]
    year = hdf[-11:-7]
    print(f"year: {year}", end="--->")
    destination_file = destination_path+'barley_median_'+year+'_qa.hdf'
    # Load  cloud masked images
    images, attrib = lut.load_data(hdf, csv)
    # compute median pixel vales of field parcel
    medians = lut.compute_median_field(images)

    nimages = len(medians)
    nbands = medians[0].shape[-1]
    with hf.File(destination_file, 'w') as f1:
        for i in np.arange(nimages):
            key = attrib.new_ID.values[i]
            maski = attrib.iloc[i, 10:].values
            med = medians[i]
            res_med = np.ones((365, nbands))*cfg.fill_value
            for k in np.arange(nbands):
                res_med[np.where(maski), k] = med[:, k]
            f1.create_dataset(str(key),
                              data=res_med,
                              dtype='f8',
                              compression="gzip",
                              compression_opts=9)
    print("\n")


(1315557, 22)

## Step 2: Apply temporal averaging on median fields created in step 1

In [ ]:
import glob
import h5py as hf
import numpy as np
import pandas as pd
import config as cfg
import utils_ls as lut

# get list of hdf image files and csv attribute files
years = cfg.years
attribute_vars = cfg.attribute_vars

hdfs = sorted(glob.glob('data/ls7/median/*.hdf'))
csvs = sorted(glob.glob('data/ls7/median/*.csv'))
nfiles = len(hdfs)
print(nfiles)
ndvi_index = -1

# aggregation parameters
day_start = 1
day_end = 365
day_span = 30
window_edges = np.arange(day_start+day_span-1, day_end, day_span)

# column names to store the aggregated data
ts_cols = ['day ' + str(x)+'-'+str(y) for x, y in
           zip(window_edges-day_span, window_edges)]
# %%
# for each hdf file (which belongs to a year, aggregate the ndvi values)
df = pd.DataFrame(columns=attribute_vars+ts_cols)
for i in np.arange(nfiles):
    year = years[i]
    print(year)
    # Load hdf and attributes data
    median_fields, attrib = lut.load_median_fields(hdfs[i], csvs[i])
    median_fields = np.transpose(np.array(median_fields), (2, 1, 0))
    vi = median_fields[ndvi_index]
    # compute masked mean across time given a matrix (nexamples, nseq)
    time_avg_ndvi = np.transpose(lut.window_mean2d(vi, window_edges))
    df_temp = pd.DataFrame(time_avg_ndvi, columns=ts_cols)
    df_temp[attribute_vars] = attrib[attribute_vars]
    df = df.append(df_temp, ignore_index=True, sort=False)
destination_file = f'ndvi_{day_start}_{day_span}_{day_end} _all_years.csv'
if 0:
    df.to_csv(destination_file, index=False)
    df = None

## Create data for area investigation
This is carried out only on years with large crop loss fields: 2004, 2008, 2012, 2015

In [ ]:
import glob
import h5py as hf
import numpy as np
import pandas as pd
import config as cfg
import utils_ls as lut

data_path = cfg.data_path + cfg.ml_ready_ndvi_data # fully processed ndvi values
fulldf = pd.read_csv(data_path)
togverysmall = pd.DataFrame()
togsmall = pd.DataFrame()
togmedium = pd.DataFrame()
toglarge = pd.DataFrame()
togverylarge = pd.DataFrame()


for year in [2004, 2008, 2012, 2015]:
    # print(year)
    df = fulldf[fulldf['year'] == year]

    verysmalllimit = 50  # 0.5ha
    smalllimit = 100  # 0.99 ha
    mediumlimit = 300  # 2.99 ha
    largelimit = 500  # 4.99ha

    small = df[df['area'] < smalllimit]
    medium = df[(df['area'] < mediumlimit) & (df['area'] >= smalllimit)]
    large = df[df['area'] >= mediumlimit]

    print(year)
    print('all')
    # print('verysmall all: ' +str(verysmall.count()[0]))
    print(small.count()[0])
    print(medium.count()[0])
    print(large.count()[0])
    # print(verylarge.count()[0])

    print('loss')
    # print('verysmall loss: ' + str(np.count_nonzero(verysmall,axis=0)[5]))
    print(np.count_nonzero(small, axis=0)[5])
    print(np.count_nonzero(medium, axis=0)[5])
    print(np.count_nonzero(large, axis=0)[5])

    togsmall = pd.concat([togsmall, small])
    toglarge = pd.concat([toglarge, large])
    togmedium = pd.concat([togmedium, medium])


# ndvi_1_30_365_all_years.csv
path = 'data/3classes/'
name = 'ndvi_1_30_365_all_years_no_ndvieq1_'
smallname = path + name + 'small.csv'
largename = path + name + 'large.csv'
mediumname = path + name + 'medium.csv'


togsmall.to_csv(smallname, index=False)
toglarge.to_csv(largename, index=False)
togmedium.to_csv(mediumname, index=False)
# togverysmall.to_csv(verysmallname,index=False)
# togverylarge.to_csv(verylargename,index=False)

# newlarge = pd.read_csv(cfg.data_path  + name + 'large.csv')

# print(np.count_nonzero(newlarge, axis=0)[5])
